In [1]:
from ilqr import iLQR
import gym
import numpy as np
import pandas as pd

from aprl.agents import MujocoFiniteDiffDynamics, MujocoFiniteDiffCost
from aprl.agents.mujoco_control import MujocoFiniteDiffDynamicsLowLevel, MujocoFiniteDiffDynamicsWarmstart

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Logging to /tmp/openai-2019-01-26-14-32-22-017106
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']
Choosing the latest nvidia driver: /usr/lib/nvidia-396, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-396']


In [2]:
# Environment setup
env = gym.make('Reacher-v2').unwrapped
env.frame_skip = 1
env.seed(21)
_obs = env.reset()

In [3]:
# Planning setup
N = 100  # planning horizon
us_init = np.array([env.action_space.sample() for _ in range(N)])

dynamics = {
    # Uses mujoco_py's MjSimState. This saves time, qpos, qvel, act and udd_state.
    #'mujoco_py': MujocoFiniteDiffDynamics(env),
    # Uses my MujocoRelevantSimState, which contains all fields that MuJoCo's derivative.cpp copies.
    #'my_all': MujocoFiniteDiffDynamicsLowLevel(env, kind='all'),
    # All fields I think matter; excludes qfrc and xfrc_applied
    #'my_recommended': MujocoFiniteDiffDynamicsLowLevel(env, kind='recommended'),
    # qpos, qvel and qacc; no warmstart.
    #'my_basic_plus': MujocoFiniteDiffDynamicsLowLevel(env, kind='basic_plus'),
    # As above, but restricts to fields qpos & qvel. 
    # I expect this to match mujoco_py, since time does not matter, and act and udd_state are blank for Reacher.
    'my_basic': MujocoFiniteDiffDynamicsLowLevel(env, kind='basic'),
    # Like my_basic, but saving qacc_warmstart.
    'my_warmstart': MujocoFiniteDiffDynamicsWarmstart(env),
}
x0s = {k: dyn.get_state() for k, dyn in dynamics.items()}

# Finite difference cost

In [ ]:
finite_cost = MujocoFiniteDiffCost(env)
finite_ilqr = iLQR(dynamics, finite_cost, N)

In [ ]:
finite_xs, finite_us = finite_ilqr.fit(x0, us_init, n_iterations=100, on_iteration=on_iteration)

# Analytic cost

In [4]:
import theano
from theano import tensor as T
from ilqr.cost import AutoDiffCost

# Reacher, Gym observation:
# obs[0:1]: xs; np.cos(qpos[0:2]) (qpos[0] is joint0, qpos[1] is joint1)
# obs[2:3]: ys; np.sin(qpos[0:2]);
# obs[4:5]: goal x and y; qpos[2:]; (target_x and target_y)
# obs[6:7]: theta dot
# obs[8:9]: xy of fingertip - target

def make_reacher_cost(kind, control_weight=1.0):
    # qpos[0:3]: theta of joint 0, theta of joint 1; target x and y.
    qpos_inputs = [T.dscalar('theta'), T.dscalar('phi'), T.dscalar('targetx'), T.dscalar('targety')]
    # qvel: derivatives of the above; note target x and y are constant so have derivative zero.
    qvel_inputs = [T.dscalar('thetadot'), T.dscalar('phidot'), T.dscalar('_zero1'), T.dscalar('_zero2')]
    # qacc: second derivatives of qpos. We don't actually use these in the cost.
    qacc_inputs = [T.dscalar('_acc{}'.format(i)) for i in range(len(qpos_inputs))]
    # qacc_warmstart: same shape as qacc
    qacc_warmstart_inputs = [T.dscalar('_accwarm{}'.format(i)) for i in range(len(qpos_inputs))]
    # qfrc_applied: same shape as qacc
    qfrc_applied_inputs = [T.dscalar('_qfrc_applied{}'.format(i)) for i in range(len(qpos_inputs))]
    # xfrc_applied: (5,6)
    xfrc_applied_inputs = [T.dscalar('_xfrc_applied{}'.format(i)) for i in range(5 * 6)]
    if kind == 'mujoco_py':
        # Reacher, MJSimState.flatten():
        # obs[0]: time step, obs[1:4]: qpos[0:3]; obs[5:8]: qvel[0:3]
        # In general might include action and udd_state, but not for Reacher.
        x_inputs = [T.dscalar('_time')] + qpos_inputs + qvel_inputs
    elif kind == 'my_all':
        # Reacher, MujocoRelevantState.flatten()
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs + qacc_warmstart_inputs + qfrc_applied_inputs + xfrc_applied_inputs
    elif kind == 'my_recommended':
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs + qacc_warmstart_inputs
    elif kind == 'my_basic_plus':
        x_inputs = qpos_inputs + qvel_inputs + qacc_inputs
    elif kind in ['my_basic', 'my_warmstart']:
        x_inputs = qpos_inputs + qvel_inputs
    else:
        raise ValueError("Unrecognised kind: '{}'".format(kind))
    u_inputs = [T.dscalar('thetadotdot'), T.dscalar('phidotdot')]
    qpos = T.stack(qpos_inputs)
    u = T.stack(u_inputs)
    
    control_cost = T.dot(u, u)
    target_xpos = qpos[2:4]
    body1_xpos = 0.1 * T.stack([T.cos(qpos[0]), T.sin(qpos[0])])
    fingertip_xpos_delta = 0.11 * T.stack([T.cos(qpos[1]), T.sin(qpos[1])])
    fingertip_xpos = body1_xpos + fingertip_xpos_delta
    delta = fingertip_xpos - target_xpos
    state_cost = T.sqrt(T.dot(delta, delta))
    l = state_cost + control_weight * control_cost
    l_terminal = T.zeros(())
    return AutoDiffCost(l, l_terminal, x_inputs, u_inputs)

In [5]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    info = "converged" if converged else ("accepted" if accepted else "failed")
    print("iteration", iteration_count, info, J_opt, xs[-1])

costs = {k: make_reacher_cost(k) for k in dynamics.keys()}
ilqrs = {k: iLQR(dyn, costs[k], N) for k, dyn in dynamics.items()}
xs = {}
us = {}
print(ilqrs.keys())
for k, ilqr in ilqrs.items():
    print('*** Fitting {} ***'.format(k))
    x0 = x0s[k]
    xs[k], us[k] = ilqr.fit(x0, us_init, n_iterations=100, on_iteration=on_iteration)

dict_keys(['my_basic', 'my_warmstart'])
*** Fitting my_basic ***
iteration -1 converged 88.51825564104233 [ 2.34154471 -1.15391508 -0.08007569  0.15573813  3.23156607 -4.03874575
  0.          0.        ]
iteration 0 accepted 19.023879164326424 [ 3.70710096  1.68786581 -0.08007569  0.15573813  3.39923414 -3.90358424
  0.          0.        ]
iteration 1 failed 19.02387916432642 [ 3.70710096  1.68786581 -0.08007569  0.15573813  3.39923414 -3.90358424
  0.          0.        ]
iteration 2 accepted 18.7338008743691 [ 3.687388    1.67741348 -0.08007569  0.15573813  3.29310985 -4.00105306
  0.          0.        ]
iteration 3 accepted 18.44113047565978 [ 3.2482365   1.14529257 -0.08007569  0.15573813  3.09107847 -3.91499404
  0.          0.        ]
iteration 4 failed 18.441130475659783 [ 3.2482365   1.14529257 -0.08007569  0.15573813  3.09107847 -3.91499404
  0.          0.        ]
iteration 5 accepted 17.331319547358124 [ 2.98258438  1.18370403 -0.08007569  0.15573813  2.61567133 -3.8249

iteration 27 accepted 11.418282620160033 [ 2.80781568  1.36664669 -0.08007569  0.15573813  2.71942022 -2.32597093
  0.          0.        ]
iteration 28 accepted 11.418080611187294 [ 2.80615888  1.37006156 -0.08007569  0.15573813  2.71458665 -2.31910516
  0.          0.        ]
iteration 29 converged 11.418078277023211 [ 2.80620064  1.36967572 -0.08007569  0.15573813  2.71448564 -2.31944682
  0.          0.        ]


# Receding horizon

In [8]:
from ilqr.controller import RecedingHorizonController

def receding(underlying):
    k = 'receding_' + underlying
    dynamics[k] = dynamics[underlying]
    x0s[k] = x0s[underlying]
    controller = RecedingHorizonController(x0s[k], ilqrs[underlying])
    rew = []
    xs[k] = []
    us[k] = []
    for x, u in controller.control(us_init, subsequent_n_iterations=10):
        ob, r, done, info = env.step(u)
        xs[k].append(x)
        us[k].append(u)
        rew.append(r)
        print('iteration', len(rew), r, x, u)
        if len(rew) == N:
            break

In [9]:
receding('my_basic')

iteration 1 -0.5536921072412293 [[-0.01698329  0.03921564 -0.08007569  0.15573813  0.00389678  0.00462729
   0.          0.        ]
 [-0.01447352  0.04437551 -0.08007569  0.15573813  0.49723578  1.0256441
   0.          0.        ]] [[0.24840052 0.51327324]]
iteration 2 -0.5142465062212788 [[-0.01447352  0.04437551 -0.08007569  0.15573813  0.49723578  1.0256441
   0.          0.        ]
 [-0.00721384  0.05943453 -0.08007569  0.15573813  0.9539405   1.98456336
   0.          0.        ]] [[0.23242534 0.4871615 ]]
iteration 3 -0.4870369566363517 [[-0.00721384  0.05943453 -0.08007569  0.15573813  0.9539405   1.98456336
   0.          0.        ]
 [ 0.00441893  0.08377842 -0.08007569  0.15573813  1.37191951  2.88271886
   0.          0.        ]] [[0.21521456 0.46140925]]
iteration 4 -0.45179140428022413 [[ 0.00441893  0.08377842 -0.08007569  0.15573813  1.37191951  2.88271886
   0.          0.        ]
 [ 0.02007956  0.11673339 -0.08007569  0.15573813  1.7595633   3.70690169
   0.      

iteration 33 -0.1582514522604642 [[ 1.0542219   2.63701703 -0.08007569  0.15573813  3.24392539 10.55337418
   0.          0.        ]
 [ 1.08583738  2.74235095 -0.08007569  0.15573813  3.07943581 10.51347725
   0.          0.        ]] [[-0.06654518  0.03272179]]
iteration 34 -0.1551750387627167 [[ 1.08583738  2.74235095 -0.08007569  0.15573813  3.07943581 10.51347725
   0.          0.        ]
 [ 1.11578902  2.84719131 -0.08007569  0.15573813  2.91116636 10.45469175
   0.          0.        ]] [[-0.069245    0.02302845]]
iteration 35 -0.14954847510140512 [[ 1.11578902  2.84719131 -0.08007569  0.15573813  2.91116636 10.45469175
   0.          0.        ]
 [ 1.1441382   2.95134931 -0.08007569  0.15573813  2.75891646 10.37703859
   0.          0.        ]] [[-0.06201247  0.01325082]]
iteration 36 -0.14568790390797418 [[ 1.1441382   2.95134931 -0.08007569  0.15573813  2.75891646 10.37703859
   0.          0.        ]
 [ 1.170926    3.02885029 -0.08007569  0.15573813  2.59886347  4.2720726

iteration 64 -0.1395774487984476 [[ 1.4235479   2.61408185 -0.08007569  0.15573813  0.09375736 -0.73091447
   0.          0.        ]
 [ 1.42450784  2.60772489 -0.08007569  0.15573813  0.09822411 -0.54079529
   0.          0.        ]] [[0.00271237 0.09189752]]
iteration 65 -0.1450443032047143 [[ 1.42450784  2.60772489 -0.08007569  0.15573813  0.09822411 -0.54079529
   0.          0.        ]
 [ 1.42556544  2.60319634 -0.08007569  0.15573813  0.11326959 -0.36520691
   0.          0.        ]] [[0.00805245 0.08554499]]
iteration 66 -0.14855566693843467 [[ 1.42556544  2.60319634 -0.08007569  0.15573813  0.11326959 -0.36520691
   0.          0.        ]
 [ 1.42664308  2.60015915 -0.08007569  0.15573813  0.10227771 -0.24243554
   0.          0.        ]] [[-0.00495973  0.05987779]]
iteration 67 -0.14261006500018628 [[ 1.42664308  2.60015915 -0.08007569  0.15573813  0.10227771 -0.24243554
   0.          0.        ]
 [ 1.42757765  2.59816424 -0.08007569  0.15573813  0.08466488 -0.15668948
  

iteration 93 -0.16020731435640018 [[ 1.4289061   2.60001375 -0.08007569  0.15573813  0.04314812  0.09561923
   0.          0.        ]
 [ 1.42921227  2.60072956 -0.08007569  0.15573813  0.01812675  0.04762201
   0.          0.        ]] [[-0.0123611  -0.02364467]]
iteration 94 -0.15617957672011815 [[ 1.42921227  2.60072956 -0.08007569  0.15573813  0.01812675  0.04762201
   0.          0.        ]
 [ 1.42933836  2.60100292 -0.08007569  0.15573813  0.00710874  0.00711897
   0.          0.        ]] [[-0.00544736 -0.02011826]]
iteration 95 -0.14751872849096465 [[ 1.42933836  2.60100292 -0.08007569  0.15573813  0.00710874  0.00711897
   0.          0.        ]
 [ 1.42931729  2.60099568 -0.08007569  0.15573813 -0.01129075 -0.00854167
   0.          0.        ]] [[-0.00921299 -0.00783516]]
iteration 96 -0.1450180917700056 [[ 1.42931729  2.60099568 -0.08007569  0.15573813 -0.01129075 -0.00854167
   0.          0.        ]
 [ 1.42924227  2.60092144 -0.08007569  0.15573813 -0.00372575 -0.006310

# Rollouts

In [6]:
import time

def rollout(env, dynamics, x0, us, render=False):
    dynamics.set_state(x0)
    if render:
        env.render()
    rew = []
    actual_xs = []
    for u in us:
        _obs, r, done, info = env.step(u)
        rew.append(r)
        actual_xs.append(dynamics.get_state())
        assert not done
        if render:
            env.render()
            time.sleep(0.02)
    return rew, actual_xs

In [11]:
rews = {}
actual_xs = {}
for k, solved_us in us.items():
    print(k)
    rews[k], actual_xs[k] = rollout(env.unwrapped, dynamics[k], x0s[k], solved_us, render=True)
rewards = {k: sum(r) for k, r in rews.items()}
lengths = {k: len(r) for k, r in rews.items()}
pd.DataFrame({'rewards': rewards, 'lengths': lengths})

my_basic
my_warmstart
receding_my_basic


,rewards,lengths
my_basic,-20.519558,100
my_warmstart,-20.519558,100
receding_my_basic,-19.313064,100
